In [ ]:
import confmain
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras.regularizers import L1L2

from util_module import util_func
from util_module import model_func

#### **Load X and Y**

In [ ]:
train_set, val_set, test_set, zpad_length = util_func.get_x_y('../data/ludb_processed/ludb_ii.pickle')

X_train, y_train = train_set
X_val, y_val = val_set
X_test, y_test = test_set

zpad_length_train, zpad_length_val, zpad_length_test = zpad_length

#### **Transfer Learning Model Hafizh - Jordan**

In [ ]:
# CHANGE ACCORDINGLY MOST OF THE THINGS HERE ARE HARD-CODED
model_info = {
    'name': 'TransferLearning_Hafizh-Jordan',
    'batch_size': 8,
    'Additional Info': 'Learning rate 0.001 for first 100 epochs and 0.00001 for the remaining 200 epochs'
    
}

model_hafizh = tf.keras.models.load_model(f'../model/base_model/hafizh.h5')
model_hafizh.trainable = False # Freeze layers

input_layer = tf.keras.layers.Input(shape=(X_train.shape[1], 1))
x = tf.keras.layers.Conv1D(8, 5, padding='same', activation='relu')(input_layer)
x = tf.keras.layers.Normalization()(x)
x = tf.keras.layers.ReLU()(x)
for layer in model_hafizh.layers[1:7]:
    x = layer(x)
x = tf.keras.layers.Dropout(rate=0.3)(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(X_train.shape[1], kernel_regularizer=L1L2(l2=0.0001), return_sequences=True))(x)
x = tf.keras.layers.Dense(512, kernel_regularizer=L1L2(l2=0.0001), activation='relu')(x)
x = tf.keras.layers.Dropout(rate=0.3)(x)
output_layer = tf.keras.layers.Dense(8, activation='softmax')(x)

new_model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
new_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history_phase_1 = new_model.fit(X_train, y_train, epochs=100, batch_size=8, validation_data=(X_val, y_val))

# Fine-tune
for layer in new_model.layers[6:10]:
    layer.trainable = True
opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
new_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history_phase_2 = new_model.fit(X_train, y_train, epochs=200, batch_size=8, validation_data=(X_val, y_val))

history_combined = {}
for key in history_phase_1.history.keys():
    history_combined[key] = history_phase_1.history[key] + history_phase_2.history[key]

In [ ]:
model_func.generate_results(new_model, history_combined, model_info, train_set, val_set, test_set, zpad_length)